# **<mark>DataBase(資料庫)<mark>**
## "DataBase"腳本能擷取StorageServer內的dicom檔數據，並建立檔案資料庫DataBase以及影像清單sorting_list
StorageServer內的dicom檔案分為由mini-PACS接收來自scanners的"dcm4chee server"內的原始資料Raw data(含個資)以及由豐康協助建立的"dicom server"的去名化影像anonymous data(無個資僅含病歷號)，兩者都可以使用"DataBase"腳本來建立檔案資料庫DataBase以及影像清單sorting_list

## 使用說明:
1.請選擇要建立<font color=green>資料庫DataBase</font>的<font color=blue>資料年月</font>和<font color=#0099ff>影像種類(原始資料庫raw或去名化資料庫anonymous)</font>  
2.DataBase腳本會建立<font color=green>資料庫DataBase</font>以及方便使用者查看資料庫內容的<font color=green>影像清單sorting_list</font>，檔案會存在<font color=green>output資料夾</font>     
3.<font color=red>DataBase腳本的執行時間較長，請詳閱注意事項</font>  

註1:因為原始資料Raw data中包含非影像檔(例如SR, PR)，因此去名化資料庫anonymous的檔按數量可能小於或等於原始資料庫raw的檔案數量，但並不影響影像本身

## <font color=red>注意事項:
<font color=red>★ 因影像資料量龐大，請定期更新資料庫以利後續資料取用  
★ 建立1個月份的資料庫約需費時**20-40小時**(視影像量而定)，建立期間可以關閉螢幕，但不可以中斷網路以及腳本，否則要重新執行  
★ 建議於次月2號後再讀取DataBase以免資料缺失  
★ 腳本請依序使用以減少錯誤: 以"DataBase"腳本建立對應的影像資料庫>>以"Filter"腳本進行篩選(以免資料量過於龐大)>>以"Download"腳本能進行下載  
★ 腳本執行後會紀錄上次的執行內容，若發生錯誤，請找到原始的檔案DataDownload.zip，解壓縮後置換換掉整個DataDownload資料夾(詳細方法請參考操作說明-Servers)  
★ 置換DataDownload資料夾前請備份您的重要資料表  
★ 腳本產生的檔案中，<font color=violet>資料庫檔案DataBase_xxx</font>是給腳本使用的資料表，而<font color=violet>影像清單sorting_list</font>則是方便給使用者查看影像內容的資料表，
    使用腳本時請務必放入正確的檔案  </font>

# **<font color=red>腳本使用方法:<p><font color=red>滑鼠選擇要執行的對話框，使對話框左邊顯示<font color=blue>藍色粗邊框<font color=red>後，按下ctrl+enter執行該對話框</font>**

# Step 1: 匯入功能包
(匯入成功會有提示)

In [1]:
#Step 1: import packages  (選取本對話框並按下ctrl+enter執行)
from utils.Functions import *
from utils.get import get_target_list,get_database,get_save_file_name  #2022/7/13 update
from utils import *

DEFAULT PARAMETERS IMPROT SCCESS!
對話框回報內容: 紅色標示注意事項；亮紫色提示類別；藍色標示input資料夾及input detail資料內容；綠色標示output資料；粉紅色背景表示程式碼錯誤
20220713 16:33:56 PACKAGES IMPORT SUCCESS!
請執行下一個步驟


# Step 2: 請在以下對話框中設定要讀取的資料年、月
(雙擊對話框可進行編輯)  
(執行Step 2對話框後，請確認參數內容，無誤則執行Step 3，參數錯誤請修改並重新執行Step 2)

In [2]:
#Step 2: 請設定要讀取的資料年、月、資料位置後按下ctrl+enter執行本對話框
Y=2022   #指定要讀取的年份
m=7      #指定要讀取的月份
Type='anonymous'   #指定讀取原始檔案(raw)或是去名化檔案(anonymous)

#底下兩行不用改
Period=str(Y*100+m)  #轉換後的期間
print(check_DB.format(BlueLight.format(dicom_point),PurpleLight.format(Period),PurpleLight.format(Type+' data')),Red.format(check_above),sep='\n')

資料路徑: ../storage
即將建立 202107 raw data 的資料庫
請確認上述內容無誤後再執行下一步驟


# Step 3: 建立資料庫，並產生DataBase以及sorting_list
(執行Step 3對話框後會出現進度條，請務必等待進度完成，結束後會有 <font color=red>執行結束!</font> 的提示)

In [7]:
#Step 3: read dicom head and build up DataBase and sorting_list  (選取本對話框並按下ctrl+enter執行)
#optional settings:
# dicom_point='../storage'    #[optional]資料路徑，請填入連結到storage server上絕對路徑/dicom的節點

#define output file names:  #2022/7/13 update
save_csv_name,save_sorting_csv_name=get_save_file_name(output_path,Suffix='_{}_{}'.format(Type,Period))
print('正在'+DB_build,waiting,'\n資料:',save_csv_name,save_sorting_csv_name)

#建立所有dicom的檔案路徑:
S=time.time()
tar_list=get_target_list(Y,m,Type=Type,Mount_point=dicom_point)  #取得檔案列表
# tar_list=tar_list[::2000]  #for test
E=time.time()
t=E-S
print(time_spend.format(total.format(len(tar_list),''),cut_float.format(1) % (t/60),'min,{}{}'.format('正在'+DB_build,waiting)))

#建立DataBase:
S=time.time()
Output_df=get_database(Target_list=tar_list,Output_template=df,Save_csv_name=save_csv_name)     
E=time.time()
t=E-S
print(time_spend.format('已完成{}'.format(DB_build),cut_float.format(2) % (t/3600),'hr'))

#建立sorting_list:  #新增column "InputFolder" (update:2022/7/13)
Input_DB=Output_df
Output_sorting_list=GetUnique(Input_DB,df_sorting)
Output_sorting_list.to_csv(save_sorting_csv_name,index=0)
print(save.format(sort_build+': {} {}'.format(Output_sorting_list.shape,GreenItalic.format(save_sorting_csv_name))),Red.format(done))

  0%|          | 0/31 [00:00<?, ?it/s]

正在建立資料庫DataBase 請稍等... 
資料: output/DataBase_raw_202107-20220713163513.csv output/sorting_list_raw_202107-20220713163513.csv


100%|██████████| 31/31 [01:47<00:00,  3.45s/it]
2it [00:00, 15.42it/s]

共有 113 個檔案，建立費時:1.8 min,正在建立資料庫DataBase請稍等...


113it [00:07, 15.15it/s]


(113, 38) output/DataBase_raw_202107-20220713163513.csv saved
已完成建立資料庫DataBase，建立費時:0.00 hr
sorting_list已建立: (107, 13) output/sorting_list_raw_202107-20220713163513.csv saved 
執行結束!


# Step 4: 請至<font color=green>output資料夾</font>查看結果

## \~底下沒東西了~

<font size="4">版本資訊: <font size="3">ver.220715</font><br><font size="2">
    2022/7/15:  
    1.調整說明文字順序  
    <br>
</font> 